In [1]:
from selenium import webdriver
import time

## Example 1

- open a new Firefox browser
- load the page at the given URL

In [10]:
browser = webdriver.Chrome(executable_path = 'C:/Users/kuria/anaconda3/Lib/site-packages/selenium/webdriver/chrome/chromedriver.exe')
browser.get('http://seleniumhq.org/')

## Example 2

- open a new Firefox browser
- load the Yahoo homepage
- search for “seleniumhq”
- close the browser

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

browser = webdriver.Chrome(executable_path = 'C:/Users/kuria/anaconda3/Lib/site-packages/selenium/webdriver/chrome/chromedriver.exe')
browser.get('http://www.yahoo.com')
assert 'Yahoo' in browser.title

elem = browser.find_element_by_name('p')  # Find the search box
elem.send_keys('seleniumhq' + Keys.RETURN)

browser.quit()

In [14]:
from selenium import webdriver
import time

url = "https://www.rentcollegepads.com/off-campus-housing/ohio-state/search"
driver = webdriver.Chrome(executable_path = 'C:/Users/kuria/anaconda3/Lib/site-packages/selenium/webdriver/chrome/chromedriver.exe')
driver.get(url)
html = driver.page_source.encode('utf-8')
page_num = 0

while True:
    try:
        loadmore = driver.find_element_by_css_selector('button[class = "btn btn-success btn-block"]')
        loadmore.click()
        page_num += 1

    except NoSuchElementException:
        print("Reached end of the page")
        break
#driver.find_elements_by_css_selector('.search-result-more-txt'):
#    driver.find_element_by_css_selector('.search-result-more-txt').click()
    page_num += 1
    print("getting page number "+str(page_num))
    time.sleep(1)

#html = driver.page_source.encode('utf-8')

getting page number 2
getting page number 4
getting page number 6
getting page number 8
getting page number 10
getting page number 12
getting page number 14
getting page number 16
getting page number 18
getting page number 20
getting page number 22
getting page number 24
getting page number 26
getting page number 28
getting page number 30
getting page number 32
getting page number 34
getting page number 36
getting page number 38
getting page number 40
getting page number 42
getting page number 44
getting page number 46
getting page number 48
getting page number 50
getting page number 52
getting page number 54
getting page number 56
getting page number 58


NameError: name 'NoSuchElementException' is not defined